![MLU Logo](../../data/MLU_Logo.png)

# <a name="0">Responsible AI - Logistic Regression </a>


This notebook shows how to build a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model to predict whether an individuals' income is $\leq$ 50k or not using US census data.

__Dataset:__ 
You will download a dataset for this exercise using [folktables](https://github.com/zykls/folktables). Folktables provides an API to download data from the American Community Survey (ACS) Public Use Microdata Sample (PUMS) files which are managed by the US Census Bureau. The data itself is governed by the terms of use provided by the Census Bureau. For more information, see the [Terms of Service](https://www.census.gov/data/developers/about/terms-of-service.html).


__ML Problem:__ 
Ultimately, the goal will be to predict whether an individual's income is above \\$50,000. We will  filter the ACS PUMS data sample to only include individuals above the age of 16, who reported usual working hours of at least 1 hour per week in the past year, and an income of at least \\$100. The threshold of \\$50,000 was chosen so that this dataset can serve as a comparable substitute to the [UCI Adult dataset](https://archive.ics.uci.edu/ml/datasets/adult). The income threshold can be changed easily to define new prediction tasks.


1. <a href="#1">Read the dataset</a>
2. <a href="#2">Data Processing</a>
    * <a href="#21">Exploratory Data Analysis</a>
    * <a href="#22">Select features to build the model</a>
    * <a href="#23">Train - Validation - Test Datasets</a>
    * <a href="#24">Data processing with Pipeline and ColumnTransformer</a>
3. <a href="#3">Train (and Tune) a Classifier</a>
4. <a href="#4">Test the Classifier</a>
5. <a href="#5">Accuracy Difference and DPPL</a>


Before building the logistic regression model, let's have a quick look at how a linear regression can be turned into a classifier.

Let's assume we want to use a feature (e.g. Class of Worker) to build a model that can predict the income class. A first step could be to plot the feature vs the model target. As class of worker is a categorical feature we introduce a little jitter to see the data points more easily.

<img src="img/fig-data.png" alt="drawing" width="400" height="300"/>

Now that we have a plot, we can fit a linear regression through the data points.

<img src="img/fig-lr.png" alt="drawing" width="400" height="300"/>

As we notice, the linear regression line extends beyond the target values. This is not ideal as we can end up making predictions that are outside the model target range (0 or 1); a linear regression can predict values in the range ($-\inf$, $+\inf$). To squish the linear regression values into a range of 0 to 1, we need to use a helper function. For this, we can use the sigmoid function:

<center>
$\frac{1}{1 + \exp(- (w_{0} + w_{1}\cdot x_{1}+ w_{2}\cdot x_{2} + ...))}$
</center>  
<p><br></p>

Wrapping the linear regression in a sigmoid function will create a so-called Logistic Regression and allow us to make binary predictions as the resulting values will be in the rage 0-1.

<img src="img/fig-lg.png" alt="drawing" width="400" height="300"/>

We can simply read out the values on the line now to get a probability for a certain individual obtaining a certain outcome. This covers the basics of Logistic Regression and it's time to code!

This notebook assumes an installation of the SageMaker kernel `.conda-mlu-rai:Python` through the `environment.yaml` file in SageMaker Sudio Labs.

In [ ]:
# Reshaping/basic libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid", {"axes.facecolor": ".9"})

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

# Operational libraries
import sys

sys.path.append("..")

# Fairness libraries
from folktables.acs import *
from folktables.folktables import *
from folktables.load_acs import *

# Jupyter(lab) libraries
import warnings

warnings.filterwarnings("ignore")

## 1. <a name="1">Read the dataset</a>
(<a href="#0">Go to top</a>)

To read in the dataset, we will be using [folktables](https://github.com/zykls/folktables) which provides access to the US Census dataset. Folktables contains predefined prediction tasks but also allows the user to specify the problem type.

The US Census dataset distinguishes between household and individuals. To obtain data on individuals, we use `ACSDataSource` with `survey=person`. The feature names for the US Census data follow the same distinction and use `P` for `person` and `H` for `household`, e.g.: `AGEP` refers to age of an individual.

In [ ]:
income_features = [
    "AGEP",  # age individual
    "COW",  # class of worker
    "SCHL",  # educational attainment
    "MAR",  # marital status
    "OCCP",  # occupation
    "POBP",  # place of birth
    "RELP",  # relationship
    "WKHP",  # hours worked per week past 12 months
    "SEX",  # sex
    "RAC1P",  # recorded detailed race code
    "PWGTP",  # persons weight
    "GCL",  # grand parents living with grandchildren
]

# Define the prediction problem and features
ACSIncome = folktables.BasicProblem(
    features=income_features,
    target="PINCP",  # total persons income
    target_transform=lambda x: x > 50000,
    group="RAC1P",
    preprocess=adult_filter,  # applies the following conditions; ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    postprocess=lambda x: x,  # applies post processing, e.g. fill all NAs
)

# Initialize year, duration ("1-Year" or "5-Year") and granularity (household or person)
data_source = ACSDataSource(survey_year="2018", horizon="1-Year", survey="person")
# Specify region (here: California) and load data
ca_data = data_source.get_data(states=["CA"], download=True)
# Apply transformation as per problem statement above
ca_features, ca_labels, ca_group = ACSIncome.df_to_numpy(ca_data)

# Convert numpy array to dataframe
df = pd.DataFrame(
    np.concatenate((ca_features, ca_labels.reshape(-1, 1)), axis=1),
    columns=income_features + [">50k"],
)

# For further modelling we want to use only 2 groups (see DATAPREP notebook for details)
df = df[df["RAC1P"].isin([6, 8])].copy(deep=True)

## 2. <a name="2">Data Processing</a>
(<a href="#0">Go to top</a>)

### 2.1 <a name="21">Exploratory Data Analysis</a>
(<a href="#2">Go to Data Processing</a>)

We look at number of rows, columns, and some simple statistics of the dataset.

In [ ]:
# Print the first five rows
# NaN means missing data
df.head()

In [ ]:
# Check how many rows and columns we have in the data frame
print("The shape of the dataset is:", df.shape)

In [ ]:
# Let's see the data types and non-null values for each column
df.info()

We can clearly see that all columns are numerical (`dtype = float64`). However, when checking the column headers (and information at top of the notebook), we should notice that we are actually dealing with multimodal data. We expect to see a mix of categorical, numerical and potentially even text information.

Let's cast the features accordingly. We start by creating list for each feature type.

In [ ]:
categorical_features = [
    "COW",
    "SCHL",
    "MAR",
    "OCCP",
    "POBP",
    "RELP",
    "SEX",
    "RAC1P",
    "GCL",
]

numerical_features = ["AGEP", "WKHP", "PWGTP"]

In [ ]:
# We cast categorical features to `category`
df[categorical_features] = df[categorical_features].astype("object")

# We cast numerical features to `int`
df[numerical_features] = df[numerical_features].astype("int")

Let's check with `.info()` again to make sure the changes took effect.

In [ ]:
df.info()

Looks good, so we can now separate model features from model target to explore them separately.

In [ ]:
model_target = ">50k"
model_features = categorical_features + numerical_features

print("Model features: ", model_features)
print("Model target: ", model_target)

In [ ]:
# Double check that that target is not accidentally part of the features
model_target in model_features

All good here. We made sure that the target is not in the feature list. If we find the above statement showing `True` we need to remove the target by calling `model_features.remove(model_target)`.

Let's have a look at missing values next.


#### Missing values
The quickest way to check for missing values is to use `.isna().sum()`. This will provide a count of how many missing values we have. In fact, we can also see the count of missing values with `.info()` as it provided a count of non-null values.

In [ ]:
# Show missing values
df.isna().sum()

Before starting with the plots, let's have a look at how many unique instances we have per column. This helps us avoid plotting charts with hundreds of unique values. Let's filter for columns with fewer than 10 unique instances.

In [ ]:
shortlist_fts = (
    df[model_features]
    .apply(lambda col: col.nunique())
    .where(df[model_features].apply(lambda col: col.nunique()) < 10)
    .dropna()
)

print(shortlist_fts)

#### Target distribution

Let's check our target distribution.

In [ ]:
df[model_target].value_counts().plot.bar(color="black")
plt.show()

We notice that we are dealing with an imbalanced dataset. This means there are more examples for one type of results (here: 0; meaning individuals earning $\leq$ 50k). This is relevant for model choice and potential up-sampling or down-sampling to balance out the classes.

#### Feature distribution(s)

Let's now plot bar charts for the shortlist features of our dataset (as per above: shortlist - feature columns with less than 10 unique instance classes).

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
fig.suptitle("Feature Bar Plots")

fts = range(len(shortlist_fts.index.tolist()))
for i, ax in zip(fts, axs.ravel()):
    df[shortlist_fts.index.tolist()[i]].value_counts().plot.bar(color="black", ax=ax)
    ax.set_title(shortlist_fts.index.tolist()[i])
plt.show()

### 2.2 <a name="22">Select features to build the model</a>
(<a href="#2">Go to Data Processing</a>)

During the extended EDA in the DATAPREP notebook, we learned that `GCL` is a feature that is equally present for both outcome types and also contains a lot of missing values. Therefore, we can drop it from the list of features we want to use for model build. We also drop `OCCP` and `POBP` as those features have too many unique categories.

In [ ]:
to_remove = ["GCL", "OCCP", "POBP"]

# Drop to_remove features from the respective list(s) - if applicable
for ft in to_remove:
    if ft in model_features:
        model_features.remove(ft)
    if ft in categorical_features:
        categorical_features.remove(ft)
    if ft in numerical_features:
        numerical_features.remove(ft)

In [ ]:
# Let's also clean up the dataframe and only keep the features and columns we need
df = df[model_features + [model_target]].copy(deep=True)

### 2.3 <a name="23">Train - Validation - Test Datasets</a>
(<a href="#2">Go to Data Processing</a>)

To get a training, test and validation set, we will use sklearn's [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

In [ ]:
train_data, test_data = train_test_split(
    df, test_size=0.1, shuffle=True, random_state=23
)
train_data, val_data = train_test_split(
    train_data, test_size=0.15, shuffle=True, random_state=23
)

# Print the shapes of the Train - Test Datasets
print(
    "Train - Test - Validation datasets shapes: ",
    train_data.shape,
    test_data.shape,
    val_data.shape,
)

### 2.4 <a name="24">Data processing with Pipeline and ColumnTransformer</a>
(<a href="#2">Go to Data Processing</a>)

Let's build a full model pipeline. We need pre-processing split per data type, and then combine everything back into a composite pipeline along with a model. To achieve this, we will use sklearns `Pipeline` and `ColumnTransformer`.

__Step 1 (set up pre-processing per data type):__
> For the numerical features pipeline, the __numerical_processor__ below, we impute missing values with the mean using sklearn's `SimpleImputer`, followed by a `MinMaxScaler` (don't have to scale features when using Decision Trees, but it's a good idea to see how to use more data transforms). If different processing is desired for different numerical features, different pipelines should be built - just like shown below for the two text features.

 > In the categorical features pipeline, the __categorical_processor__ below, we impute with a placeholder value and encode with sklearn's `OneHotEncoder`. If computing memory is an issue, it is a good idea to check categoricals' unique values, to get an estimate of many dummy features will be created by one-hot encoding. Note the __handle_unknown__ parameter that tells the encoder to ignore (rather than throw an error for) any unique value that might show in the validation/and or test set that was not present in the initial training set.
 
__Step 2 (combining pre-processing methods into a transformer):__ 
 > The selective preparations of the dataset features are then put together into a collective `ColumnTransformer`, to be finally used in a Pipeline along with an estimator. This ensures that the transforms are performed automatically on the raw data when fitting the model and when making predictions, such as when evaluating the model on a validation dataset via cross-validation or making predictions on a test dataset in the future.
   
__Step 3 (combining transformer with a model):__ 
> Combine `ColumnTransformer` from Step 2 with a selected algorithm in a new pipeline. For example, the algorithm could be a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for classification problems.

In [ ]:
### STEP 1 ###
##############

# Preprocess the numerical features
numerical_processor = Pipeline(
    [("num_imputer", SimpleImputer(strategy="mean")), ("num_scaler", MinMaxScaler())]
)
# Preprocess the categorical features
categorical_processor = Pipeline(
    [
        ("cat_imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("cat_encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

### STEP 2 ###
##############

# Combine all data preprocessors from above
data_processor = ColumnTransformer(
    [
        ("numerical_processing", numerical_processor, numerical_features),
        ("categorical_processing", categorical_processor, categorical_features),
    ]
)

### STEP 3 ###
##############

# Pipeline desired all data transformers, along with an estimator at the end
# Later you can set/reach the parameters using the names issued - for hyperparameter tuning, for example
pipeline = Pipeline(
    [
        ("data_processing", data_processor),
        ("lg", LogisticRegression(solver="lbfgs", penalty=None)),
    ]
)

# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config

set_config(display="diagram")
pipeline

## 3. <a name="3">Train a Classifier</a>
(<a href="#0">Go to top</a>)

We use the pipeline with the desired data transformers, along with a Logistic Regression estimator for training.


### Model Training

We train the classifier with`.fit()` on our training dataset. 

In [ ]:
# Get train data to train the classifier
X_train = train_data[model_features]
y_train = train_data[model_target]

# Fit the classifier to the train data
# Train data going through the Pipeline is imputed (with means from the train data),
#   scaled (with the min/max from the train data),
#   and finally used to fit the model
pipeline.fit(X_train, y_train)

We won't tune the classifier at this point, and simply use the validation set as additional test:

In [ ]:
# Get validation data to tune the classifier
X_val = val_data[model_features]
y_val = val_data[model_target]

y_val_pred = pipeline.predict(X_val)

print("Model performance on the validation set:")
print("Validation accuracy:", accuracy_score(y_val, y_val_pred))

## 4. <a name="4">Test the Classifier</a>
(<a href="#0">Go to top</a>)

Let's now evaluate the performance of the trained classifier on the test dataset. We use `.predict()` this time. 


In [ ]:
# Get test data to evaluate the performance of the classifier
X_test = test_data[model_features]
y_test = test_data[model_target]

# Use the fitted model to make predictions on the test dataset
# Test data going through the Pipeline is imputed (with means from the train data),
#   scaled (with the min/max from the train data),
#   and finally used to make predictions
test_predictions = pipeline.predict(X_test)

print("Model performance on the test set:")
print("Test accuracy:", accuracy_score(y_test, test_predictions))

## 5. <a name="5">Accuracy Difference and DPPL</a>
(<a href="#0">Go to top</a>)

### DPPL
DPPL (Difference in Proportion of Predicted Labels) is an extension of DPL where the outcome we compare is the prediction the model creates (rather than the ground truth value). The equation remains basically the same, only we are counting positive outcomes as per model prediction:


$\large DPPL = \frac{\hat{n}_{pred>50k \wedge RAC1P=6}}{n_{RAC1P=6}} - \frac{\hat{n}_{pred>50k \wedge RAC1P=8}}{n_{RAC1P=8}}$

To calculate DPPL more easily, let's write a function for it that can take different parameters.

In [ ]:
def dpl(sensitive_attribute_name, attr_val, target, dataframe):
    """Function to calculate DPL or DPPL (depending on specified target)."""
    for val in attr_val:
        globals()[f"n_pos_gr{val}"] = len(
            dataframe[
                (dataframe[target] == 1) & (dataframe[sensitive_attribute_name] == val)
            ]
        )
        globals()[f"n_gr{val}"] = len(
            dataframe[dataframe[sensitive_attribute_name] == val]
        )

    dpl = n_pos_gr6 / n_gr6 - n_pos_gr8 / n_gr8
    return dpl

In [ ]:
# Create dataframe that contains predictions and the sensitive attribute
dpl_df = pd.concat(
    [
        test_data.reset_index(drop=True)[["RAC1P", ">50k"]],
        pd.Series(test_predictions, name="y_test_pred"),
    ],
    axis=1,
)

dpl("RAC1P", [6, 8], "y_test_pred", dpl_df)

Let's compare this to the original DPL value (the difference in proportion of labels in the original dataframe):

In [ ]:
dpl("RAC1P", [6, 8], ">50k", test_data)

The DPL value is smaller than the DPPL value. It seems that the model is producing a more biased output compared to the ground truth data. This could be due to the different success rates for the different groups in our model. Generally, models will make better predictions for the larger group (meaning, the larger group drives the overall model performance and whatever outcome is dominant in the larger group, will occur more frequently in the predictions).

Let's see if we can observe this behavior in our model predictions and calculate overall accuracy first.

In [ ]:
# Accuracy score across all groups
accuracy_score(dpl_df[">50k"], dpl_df["y_test_pred"])

Let's also have a look at the accuracy difference between `RAC1P=6` and `RAC1P=8`.

In [ ]:
# Accuracy score for RAC1P=1
acc_gr6 = accuracy_score(
    dpl_df[dpl_df["RAC1P"] == 6][">50k"], dpl_df[dpl_df["RAC1P"] == 6]["y_test_pred"]
)

acc_gr6

In [ ]:
# Accuracy score for RAC1P=8
acc_gr8 = accuracy_score(
    dpl_df[dpl_df["RAC1P"] == 8][">50k"], dpl_df[dpl_df["RAC1P"] == 8]["y_test_pred"]
)

acc_gr8

### Accuracy Difference
We calculate the Accuracy Difference by subtracting the two values we calculated above:

In [ ]:
acc_gr6 - acc_gr8

As expected, the accuracy for the group we assume to be at an advantage is higher than the accuracy for the disfavored group. We can dive deeper to see what's going on and plot the confusion matrix and we should also look at confidence intervals.

In [ ]:
# Confusion matrix for RAC1P=8
confusion_matrix(
    dpl_df[dpl_df["RAC1P"] == 8][">50k"], dpl_df[dpl_df["RAC1P"] == 8]["y_test_pred"]
).ravel()

Let's compare this confusion matrix with the one for the favored group.

In [ ]:
# Confusion matrix for RAC1P=1
confusion_matrix(
    dpl_df[dpl_df["RAC1P"] == 6][">50k"], dpl_df[dpl_df["RAC1P"] == 6]["y_test_pred"]
).ravel()

To create confidence intervals, we need to create samples first. We can then repeatedly calculate the metric that interests us from the new sample and eventually summarize in aggregate (or visualize in e.g., box plots).

In [ ]:
def sampling(n_iter=10, group=1):
    tn_ls, fp_ls, fn_ls, tp_ls = [], [], [], []
    output = {}
    for i in range(0, n_iter):
        sample = resample(test_data, replace=True, n_samples=5551, stratify=None)
        sample = sample[sample["RAC1P"] == group]
        tn, fp, fn, tp = confusion_matrix(
            sample[model_target], pipeline.predict(sample[model_features])
        ).ravel()
        tn_ls.append(tn), fp_ls.append(fp), fn_ls.append(fn), tp_ls.append(tp)
    output["tn"] = tn_ls
    output["fp"] = fp_ls
    output["fn"] = fn_ls
    output["tp"] = tp_ls
    return pd.DataFrame.from_dict(output, orient="columns", dtype=None, columns=None)

In [ ]:
%matplotlib inline
# Initialize figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Set title of figure
fig.suptitle("Comparison of Classification Errors")

# Set title
ax1.title.set_text("Group 6")
ax2.title.set_text("Group 8")

# Create boxplot
sns.boxplot(data=sampling(n_iter=100, group=6), ax=ax1)
sns.boxplot(data=sampling(n_iter=100, group=8), ax=ax2)

# Align y-axis
ax1.sharey(ax2)

Generally we can see that group 6 has a much higher share of positive predictions than group 8 (add FP + TP for total positive predictions). Furthermore, we observe that the errors are not balanced between false positives and false negatives for the disfavored group (whereas they are within the error bounds for the favored group). For group 8 there is a larger share of false negatives (predicted salary as $\leq$ 50k, when it was actually higher) than false positives. 
This is concerning:
- The model is penalizing individuals that should have been predicted as positive $\rightarrow$ if this model is used to show job ads based on a 50k salary threshold, a lot of individuals that should have been eligible to see the ad, won't receive it. This causes a negative reinforcement of existing bias.
- The imbalance is not as drastic for the favored group. For this group, we observe more false positives, than false negatives. This means that people are incorrectly predicted positive for the favored group. This causes further amplification of the bias.

In [ ]:
%matplotlib inline
# Initialize figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Set title of figure
fig.suptitle("Comparison of Model Predictions and Baseline")

# Set title
ax1.title.set_text("Baseline target distribution")
ax2.title.set_text("Prediction distribution")

# Create plots
dpl_df.groupby(["RAC1P", model_target]).size().unstack().plot(
    kind="bar", stacked=True, color=sns.husl_palette(2), ax=ax1
)
dpl_df.groupby(["RAC1P", "y_test_pred"]).size().unstack().plot(
    kind="bar", stacked=True, color=sns.husl_palette(2), ax=ax2
)
# Align y-axis
ax2.sharey(ax1)

We notice that the predictions are less favorable for the disfavored group with even more '0' outcomes than the baseline (which itself was already very biased).

This is the end of the notebook.